In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle
import os

base_path = "/projects/academic/smuldoon/bengieru/Community_Detection/general_diagnostics_00/"

import sys
sys.path.insert(0, base_path)

from helpers import *
from Temporal_Community_Detection import temporal_network

In [ ]:
data_path = base_path + 'M_ESCR/'

path = data_path + 'MMM/'

with open(data_path + 'spikes.pkl', 'rb') as handle:
    spikes = pickle.load(handle)
    
with open(data_path + 'comm_size.pkl', 'rb') as handle:
    comm_sizes = pickle.load(handle)
    
num_neurons = sum(comm_sizes[0])
layers = 2

window_size = 1000 # size, in frames, each adjacency matrix correspond to. better to be equal to bin_size 
standard_dev = 1.2 # for gaussian kernel
k = 5 #for jittering the spikes
pad = True

In [ ]:
binned_spikes = bin_time_series(spikes, window_size, gaussian = True, sigma = standard_dev)

adjacency_matrices = []
for i in range(layers):
    adjacency_matrices.append(cross_correlation_matrix(binned_spikes[i])[0])
    
if pad:
    padded_adjacencies = [adjacency_matrices[0]]  + adjacency_matrices + [adjacency_matrices[-1]]
    layers = layers + 2

In [ ]:
TN = temporal_network(num_neurons, 
                      layers, 
                      window_size, 
                      data = 'list__adjacency', 
                      list_adjacency = padded_adjacencies, 
                      omega = 1, 
                      kind = 'ordinal')

fig,ax = plt.subplots(1,1, figsize = (25,15))
TN.raster_plot(spikes, ax)
plt.savefig(path + 'raster_plot.pdf')

In [ ]:
grid = 51

resos = np.linspace(0, 1.5, grid)
inters = np.linspace(0, 1.5, grid)

path_no_update = path + 'no_update/'

os.makedirs(path_no_update, exist_ok = True)

In [ ]:
membership_no_update, labels_no_update = TN.run_community_detection('MMM', 
                                                                    interlayers = inters, 
                                                                    resolutions = resos)
with open(path_no_update + "MMM_labels_no_update.pkl", "wb") as fp:
    pickle.dump(labels_no_update, fp)

In [ ]:
index = 10
for m in range(5):
    for n in range(5):
        fig,ax = plt.subplots(index,index, figsize = (index*15+5,index*15))
        for i in range(index):
            for j in range(index):
                comms, c = TN.community(membership_no_update['interlayer=%.3f'%inters[i+m*index]][j+n*index], ax[i][j])
                ax[i][j].set_xticks([i for i in range(layers)])
                ax[i][j].set_yticks([i*10 for i in range(int(num_neurons/10)+1)])
                ax[i][j].tick_params(axis = 'both', labelsize = 12)
                ax[i][j].set_xlabel('Layers (Time)', fontsize = 25)
                ax[i][j].set_ylabel('Neuron ID', fontsize = 25)
                ax[i][j].set_title('%d Communities, interlayer:%.3f, resolution:%.3f'%(len(c),inters[i+m*index],resos[j+n*index]), fontsize=29)
        plt.tight_layout()
        plt.savefig(path_no_update + 'communities_%d.pdf'%((m*5)+n))

In [ ]:
information_recovery(labels_no_update, comm_sizes, 'Scattered', inters, resos, 'merge')
plt.savefig(path_no_update + 'info_recovery_scattered.pdf')

In [ ]:
information_recovery(labels_no_update, comm_sizes, 'Integrated', inters, resos, 'merge')
plt.savefig(path_no_update + 'info_recovery_integrated.pdf')

In [ ]:
path_local_update = path + 'local_update/'

os.makedirs(path_local_update, exist_ok = True)

In [ ]:
membership_local_update, labels_local_update = TN.run_community_detection('MMM', 
                                                                          update_method = 'local', 
                                                                          interlayers = inters, 
                                                                          resolutions = resos, 
                                                                          spikes = spikes)
with open(path_local_update + "MMM_labels_local_update.pkl", "wb") as fp:
    pickle.dump(labels_local_update, fp)

In [ ]:
index = 10
for m in range(5):
    for n in range(5):
        fig,ax = plt.subplots(index,index, figsize = (index*15+5,index*15))
        for i in range(index):
            for j in range(index):
                comms, c = TN.community(membership_local_update['interlayer=%.3f'%inters[i+m*index]][j+n*index], ax[i][j])
                ax[i][j].set_xticks([i for i in range(layers)])
                ax[i][j].set_yticks([i*10 for i in range(int(num_neurons/10)+1)])
                ax[i][j].tick_params(axis = 'both', labelsize = 12)
                ax[i][j].set_xlabel('Layers (Time)', fontsize = 25)
                ax[i][j].set_ylabel('Neuron ID', fontsize = 25)
                ax[i][j].set_title('%d Communities, interlayer:%.3f, resolution:%.3f'%(len(c),inters[i+m*index],resos[j+n*index]), fontsize=29)
        plt.tight_layout()
        plt.savefig(path_local_update + 'communities_%d.pdf'%((m*5)+n))

In [ ]:
information_recovery(labels_local_update, comm_sizes, 'Scattered', inters, resos, 'merge')
plt.savefig(path_local_update + 'info_recovery_scattered.pdf')

In [ ]:
information_recovery(labels_local_update, comm_sizes, 'Integrated', inters, resos, 'merge')
plt.savefig(path_local_update + 'info_recovery_integrated.pdf')

In [ ]:
path_global_update = path + 'global_update/'

os.makedirs(path_global_update, exist_ok = True)

In [ ]:
membership_global_update, labels_global_update = TN.run_community_detection('MMM', 
                                                                            update_method = 'global', 
                                                                            interlayers = inters, 
                                                                            resolutions = resos, 
                                                                            spikes = spikes)
with open(path_global_update + "MMM_labels_global_update.pkl", "wb") as fp:
    pickle.dump(labels_global_update, fp)

In [ ]:
index = 10
for m in range(5):
    for n in range(5):
        fig,ax = plt.subplots(index,index, figsize = (index*15+5,index*15))
        for i in range(index):
            for j in range(index):
                comms, c = TN.community(membership_global_update['interlayer=%.3f'%inters[i+m*index]][j+n*index], ax[i][j])
                ax[i][j].set_xticks([i for i in range(layers)])
                ax[i][j].set_yticks([i*10 for i in range(int(num_neurons/10)+1)])
                ax[i][j].tick_params(axis = 'both', labelsize = 12)
                ax[i][j].set_xlabel('Layers (Time)', fontsize = 25)
                ax[i][j].set_ylabel('Neuron ID', fontsize = 25)
                ax[i][j].set_title('%d Communities, interlayer:%.3f, resolution:%.3f'%(len(c),inters[i+m*index],resos[j+n*index]), fontsize=29)
        plt.tight_layout()
        plt.savefig(path_global_update + 'communities_%d.pdf'%((m*5)+n))

In [ ]:
information_recovery(labels_global_update, comm_sizes, 'Scattered', inters, resos, 'merge')
plt.savefig(path_global_update + 'info_recovery_scattered.pdf')

In [ ]:
information_recovery(labels_global_update, comm_sizes, 'Integrated', inters, resos, 'merge')
plt.savefig(path_global_update + 'info_recovery_integrated.pdf')

In [ ]:
path_nbr_update = path + 'nbr_update/'

os.makedirs(path_nbr_update, exist_ok = True)

In [ ]:
membership_neighborhood_update, labels_neighborhood_update = TN.run_community_detection('MMM', 
                                                                            update_method = 'neighborhood', 
                                                                            interlayers = inters, 
                                                                            resolutions = resos, 
                                                                            spikes = spikes)
with open(path_nbr_update + "MMM_labels_neighborhood_update.pkl", "wb") as fp:
    pickle.dump(labels_neighborhood_update, fp)

In [ ]:
index = 10
for m in range(5):
    for n in range(5):
        fig,ax = plt.subplots(index,index, figsize = (index*15+5,index*15))
        for i in range(index):
            for j in range(index):
                comms, c = TN.community(membership_neighborhood_update['interlayer=%.3f'%inters[i+m*index]][j+n*index], ax[i][j])
                ax[i][j].set_xticks([i for i in range(layers)])
                ax[i][j].set_yticks([i*10 for i in range(int(num_neurons/10)+1)])
                ax[i][j].tick_params(axis = 'both', labelsize = 12)
                ax[i][j].set_xlabel('Layers (Time)', fontsize = 25)
                ax[i][j].set_ylabel('Neuron ID', fontsize = 25)
                ax[i][j].set_title('%d Communities, interlayer:%.3f, resolution:%.3f'%(len(c),inters[i+m*index],resos[j+n*index]), fontsize=29)
        plt.tight_layout()
        plt.savefig(path_nbr_update + 'communities_%d.pdf'%((m*5)+n))

In [ ]:
information_recovery(labels_neighborhood_update, comm_sizes, 'Scattered', inters, resos, 'merge')
plt.savefig(path_nbr_update + 'info_recovery_scattered.pdf')

In [ ]:
information_recovery(labels_neighborhood_update, comm_sizes, 'Integrated', inters, resos, 'merge')
plt.savefig(path_nbr_update + 'info_recovery_integrated.pdf')